In [1]:
import pandas as pd

### Import and modify the dataset

In [2]:
county_df = pd.read_csv("./data/merged_county_data.csv", dtype={"fips_column": str})

#### Group Industries

In [3]:
# Group: Resource Extraction
county_df["pct_employed_resource_extraction"] = county_df[["pct_employed_agriculture_forestry_fishing_hunting_mining"]].sum(axis=1)

# Group: Manufacturing and Construction
county_df["pct_employed_manufacturing_construction"] = county_df[["pct_employed_construction", "pct_employed_manufacturing"]].sum(axis=1)

# Group: Trade and Transportation
county_df["pct_employed_trade_transportation_utilities"] = county_df[["pct_employed_wholesale_trade", 
                                                                      "pct_employed_retail_trade", 
                                                                      "pct_employed_transportation_warehousing_utilities"]].sum(axis=1)

# Group: Knowledge Based
county_df["pct_employed_knowledge_professional"] = county_df[["pct_employed_information", 
                                                               "pct_employed_finance_insurance_real_estate", 
                                                               "pct_employed_professional_scientific_management_administration"]].sum(axis=1)

# Group: Essential (Public and Social)
county_df["pct_employed_public_social"] = county_df[["pct_employed_education_healthcare_social_assistance", 
                                                      "pct_employed_public_administration"]].sum(axis=1)

# Group: Recreation and Other
county_df["pct_employed_hospitality_other"] = county_df[["pct_employed_arts_entertainment_recreation_food_accommodation", 
                                                          "pct_employed_other_services"]].sum(axis=1)                                                          

In [ ]:
# Round newly created columns to 4 decimals

county_rounding_dict = {
    "pct_employed_resource_extraction": 4,
    "pct_employed_manufacturing_construction": 4,
    "pct_employed_trade_transportation_utilities": 4,
    "pct_employed_knowledge_professional": 4,
    "pct_employed_public_social": 4,
    "pct_employed_hospitality_other": 4,
}

county_df = county_df.round(county_rounding_dict)

#### Drop Columns

In [5]:
columns_to_drop = [
    "households",
    "mean_household_size",
    "pct_employed_agriculture_forestry_fishing_hunting_mining",
    "pct_employed_construction",
    "pct_employed_manufacturing",
    "pct_employed_wholesale_trade",
    "pct_employed_retail_trade",
    "pct_employed_transportation_warehousing_utilities",
    "pct_employed_information",
    "pct_employed_finance_insurance_real_estate",
    "pct_employed_professional_scientific_management_administration",
    "pct_employed_education_healthcare_social_assistance",
    "pct_employed_public_administration",
    "pct_employed_arts_entertainment_recreation_food_accommodation",
    "pct_employed_other_services"
]

county_df.drop(columns = columns_to_drop, inplace=True)

#### Save and retrieve dataset

Variable Explainations as a table

In [6]:
# Save to csv "county_data.csv"
county_df.to_csv("./data/county_data.csv", index=False)

# Retrieve dataset
county_df = pd.read_csv("./data/county_data.csv", dtype={"fips_column": str})

county_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3144 entries, 0 to 3143
Data columns (total 34 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   fips_code                                    3144 non-null   int64  
 1   county_name                                  3144 non-null   object 
 2   state_name                                   3144 non-null   object 
 3   rucc                                         3144 non-null   int64  
 4   area_classification                          3144 non-null   object 
 5   latitude                                     3144 non-null   float64
 6   longitude                                    3144 non-null   float64
 7   population                                   3144 non-null   int64  
 8   median_age                                   3144 non-null   float64
 9   birth_rate                                   3144 non-null   float64
 10  